In [1]:
import pandas as pd

In [34]:
#read in the file
df = pd.read_csv('contacts.csv') #, encoding='iso-8859-1')

#first names are the index, so reset it and pull the right columns
df.reset_index(inplace=True)
df = df[['index', 'Middle Name', 'Home Phone 2', 'Primary Phone']]
df.columns = ['First', 'Last', 'Phone', 'Phone alt']

In [35]:
import re

In [36]:
#drop any row without a number
df = df[(df['Phone'].notnull()) | df['Phone alt'].notnull()]

In [37]:
#number cleaning
def clean_number(row):
    #check to see which number exists, default to Phone
    p = row['Phone']
    p_alt = row['Phone alt']
    if pd.isnull(p):
        p = p_alt
    
    
    #scrub it for just numbers
    cp = re.sub('[^0-9]', '', p)
    if len(cp) == 11: #includes preceding 1
        return cp[1:]
    if len(cp) > 10: #weird numbers
        return 'weird'
    return cp

In [38]:
#clean up the numbers
df['Phone Number'] = df.apply(clean_number, axis=1)

In [41]:
#remove nonUS, 800 numbers
df = df[df['Phone Number']!='weird'] 

#drop any row without a first or last name
df = df[(df['First'].notnull()) | (df['Last'].notnull())]

In [42]:
import math

In [126]:
limit = 100
nrows = df.shape[0]
iterations = math.ceil(nrows/limit)

for i in range(iterations):
    end = limit * (i+1)
    if nrows < end:
        end = nrows

    my_df = df.iloc[i*limit:end]
    my_df = my_df[['First', 'Last', 'Phone Number']]
    file = 'cleaned contacts - ' + str(i+1) + '.csv'
    my_df.to_csv(file, index=False)